In [34]:
import pandas as pd
import json
import unicodedata
import re
from collections import Counter

In [35]:
# Config chemins

POLLUTION_JSON_PATH = "pollution.json"
GREEN_CSV_PATH = "green_areas.csv"

In [36]:
# Helpers de nettoyage / normalisation

def normalize_str(s):
    """minuscule + trim + enlever NBSP + normaliser accents + compacter espaces"""
    if pd.isna(s):
        return None
    s = str(s)
    s = s.replace("\xa0", " ")     # NBSP -> espace
    s = s.strip()
    # Normalisation unicode + suppression des accents
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    # Remplacer espaces multiples par un seul
    s = re.sub(r"\s+", " ", s)
    return s.lower()

COUNTRY_MAP = {
    "russian federation": "russia",
    "united states of america": "united states",
    "united states": "united states",
    "u.s.a.": "united states",
    "great britain": "united kingdom",
    "uk": "united kingdom",
    # ajouter d'autres cas si on en voit
}

def harmonize_country_norm(cn):
    if cn is None:
        return None
    return COUNTRY_MAP.get(cn, cn)

In [37]:
# Charger pollution (JSON facelift)

with open(POLLUTION_JSON_PATH, encoding="utf-8") as f:
    pollution = json.load(f)

df_poll = pd.json_normalize(pollution["measurements"])

# Normalisation pays / villes pour jointure
df_poll["country_norm"] = df_poll["country_name"].apply(normalize_str)
df_poll["city_norm"] = df_poll["city_name"].apply(normalize_str)
df_poll["country_join"] = df_poll["country_norm"].apply(harmonize_country_norm)

print("=== Pollution JSON ===")
print("Nb de lignes :", len(df_poll))
print(df_poll[["country_name", "city_name"]].head(), "\n")

=== Pollution JSON ===
Nb de lignes : 23035
         country_name         city_name
0  Russian Federation        Praskoveya
1              Brazil  Presidente Dutra
2               Italy   Priolo Gargallo
3              Poland         Przasnysz
4              France          Punaauia 



In [38]:
# Charger green areas (CSV ou XLSX)

df_green = pd.read_csv(GREEN_CSV_PATH, encoding="utf-8-sig")

print("=== Green areas (raw) ===")
print("Nb de lignes :", len(df_green))
print("Colonnes :", list(df_green.columns), "\n")

# Renommer quelques colonnes clés pour simplifier
# (adapte les noms si pandas a fait des trucs bizarres)
col_country = "Country or Territory Name"
col_citycode = "City Code"
col_cityname = "City Name"
col_share2020 = "Average share of green area in city/ urban area 2020 (%)"
col_percap2020 = "Green area per capita 2020 (m2/person)"

# Normalisation pays / villes / code ville
df_green["country_norm"] = df_green[col_country].apply(normalize_str)
df_green["city_norm"] = df_green[col_cityname].apply(normalize_str)
df_green["citycode_norm"] = df_green[col_citycode].apply(normalize_str)
df_green["country_join"] = df_green["country_norm"].apply(harmonize_country_norm)

print("Aperçu colonnes normalisées (green):")
print(df_green[[col_country, col_cityname, col_citycode, "country_norm", "city_norm", "citycode_norm", "country_join"]].head(), "\n")


=== Green areas (raw) ===
Nb de lignes : 667
Colonnes : ['Country or Territory Name', 'City Code', 'City Name', 'SDG Sub-Region', 'SDG Region', 'Average share of green area in city/urban area 1990 (%)', 'Average share of green area in city/ urban area 2000 (%)', 'Average share of green area in city/ urban area 2010 (%)', 'Average share of green area in city/ urban area 2020 (%)', 'Green area per capita 1990 (m2/person)', 'Green area per capita 2000 (m2/person)', 'Green area per capita 2010 (m2/person)', 'Green area per capita 2020 (m2/person)', 'Data Source', 'FootNote'] 

Aperçu colonnes normalisées (green):
  Country or Territory Name       City Name          City Code country_norm  \
0               Afghanistan           Kabul           AF_KABUL  afghanistan   
1               Afghanistan           Herat           AF_HERAT  afghanistan   
2               Afghanistan  Mazar-e Sharif  AF_MAZAR_E_SHARIF  afghanistan   
3               Afghanistan        Kandahar        AF_KANDAHAR  afg

In [43]:
# A) Doublons green areas

print("=== A) Doublons GREEN AREAS ===")

# 1) Doublons sur City Name seul (normalisé)
dups_city = df_green[df_green.duplicated(subset=["city_norm"], keep=False)]
print("\nNb de lignes avec city_norm dupliqué :", len(dups_city))

# 2) Doublons sur (Country+City)
dups_country_city = df_green[df_green.duplicated(subset=["country_join", "city_norm"], keep=False)]
print("Nb de lignes avec (country_join, city_norm) dupliqués :", len(dups_country_city))

# 3) Doublons sur City Code
dups_citycode = df_green[df_green.duplicated(subset=["citycode_norm"], keep=False)]
print("Nb de lignes avec citycode_norm dupliqué :", len(dups_citycode))

# 4) Top 10 City Names ambigus (présents dans plusieurs pays)
ambig = (
    df_green.groupby("city_norm")["country_join"]
    .nunique()
    .sort_values(ascending=False)
)
ambig = ambig[ambig > 1].head(10)
print("\nTop 10 City Names ambigus (nb de pays distincts) :")
print(ambig, "\n")

# doublons de City Code en ignorant les valeurs manquantes
tmp = df_green[df_green["citycode_norm"].notna()]
dups_citycode_real = tmp[tmp.duplicated(subset=["citycode_norm"], keep=False)]
print("Doublons City Code (hors NA) =", len(dups_citycode_real))

=== A) Doublons GREEN AREAS ===

Nb de lignes avec city_norm dupliqué : 14
Nb de lignes avec (country_join, city_norm) dupliqués : 0
Nb de lignes avec citycode_norm dupliqué : 8

Top 10 City Names ambigus (nb de pays distincts) :
city_norm
hyderabad              2
london                 2
tarabulus (tripoli)    2
Name: country_join, dtype: int64 

Doublons City Code (hors NA) = 0


In [ ]:
# B) Valeurs manquantes

print("=== B) Valeurs manquantes ===")

# GREEN AREAS : remplacer chaînes vides par NaN pour la détection
cols_check_green = [
    col_country,
    col_cityname,
    col_citycode,
    col_share2020,
    col_percap2020,
]

df_green[cols_check_green] = df_green[cols_check_green].replace(
    r"^\s*$", pd.NA, regex=True
)

print("\nGreen areas - nb de valeurs manquantes :")
print(df_green[cols_check_green].isna().sum())

# POLLUTION : vérifier quelques colonnes importantes
cols_check_poll = [
    "country_name",
    "city_name",
    "aqi.value",
    "iaqi.pm25.value",
]

print("\nPollution JSON - nb de valeurs manquantes :")
print(df_poll[cols_check_poll].isna().sum())


=== B) Valeurs manquantes ===

Green areas - nb de valeurs manquantes :
Country or Territory Name                                   0
City Name                                                   8
City Code                                                   8
Average share of green area in city/ urban area 2020 (%)    0
Green area per capita 2020 (m2/person)                      0
dtype: int64

Pollution JSON - nb de valeurs manquantes :
country_name       0
city_name          0
aqi.value          0
iaqi.pm25.value    0
dtype: int64


In [41]:
# C) Doublons aussi côté pollution (pour info)

print("\n=== C) Doublons POLLUTION ===")

dups_poll_city = df_poll[df_poll.duplicated(subset=["city_norm"], keep=False)]
print("Nb de lignes pollution avec city_norm dupliqué :", len(dups_poll_city))

dups_poll_country_city = df_poll[df_poll.duplicated(subset=["country_join", "city_norm"], keep=False)]
print("Nb de lignes pollution avec (country_join, city_norm) dupliqués :", len(dups_poll_country_city))



=== C) Doublons POLLUTION ===
Nb de lignes pollution avec city_norm dupliqué : 0
Nb de lignes pollution avec (country_join, city_norm) dupliqués : 0


In [42]:
# Résumé rapide

print("\n=== Résumé rapide ===")
print(f"Green areas : {len(df_green)} lignes")
print(f"Pollution   : {len(df_poll)} lignes")
print("-> Voir les stats ci-dessus pour les doublons et valeurs manquantes.")


=== Résumé rapide ===
Green areas : 667 lignes
Pollution   : 23035 lignes
-> Voir les stats ci-dessus pour les doublons et valeurs manquantes.


In [44]:
pairs_green = set(zip(df_green["country_join"], df_green["city_norm"]))
pairs_poll  = set(zip(df_poll["country_join"],  df_poll["city_norm"]))

common = pairs_green & pairs_poll
print("Green pairs:", len(pairs_green))
print("Poll pairs :", len(pairs_poll))
print("Common     :", len(common))

# exemples de matches
print(list(common)[:20])

# si common est faible, voir ce qui manque côté green
only_green = pairs_green - pairs_poll
print("Exemples green non trouvés dans pollution:", list(only_green)[:20])

Green pairs: 667
Poll pairs : 23035
Common     : 365
[('india', 'anand'), ('france', 'lille'), ('argentina', 'san juan'), ('afghanistan', 'herat'), ('democratic republic of the congo', 'kisangani'), ('ukraine', 'odesa'), ('poland', 'legnica'), ('spain', 'burgos'), ('niger', 'niamey'), ('brazil', 'salvador'), ('honduras', 'san pedro sula'), ('united states', 'raleigh'), ('saudi arabia', 'jiddah'), ('senegal', 'ziguinchor'), ('pakistan', 'larkana'), ('namibia', 'windhoek'), ('malaysia', 'ipoh'), ('china', 'zhuji'), ('united states', 'las vegas'), ('india', 'amritsar')]
Exemples green non trouvés dans pollution: [('turkiye', 'ankara'), ('germany', 'oldenburg (oldenburg)'), ('saudi arabia', 'ad-dammam'), ('iran (islamic republic of)', 'nishabur (nishapur/neyshabur)'), ('tunisia', 'susah (sousse)'), ('dominican republic', 'santo domingo'), ('luxembourg', 'luxembourg (letzebuerg)'), ('tajikistan', 'istaravsan (istarawshan)'), ('brazil', 'parauapebas'), ('morocco', 'sidi slimane'), ('turkiye'